In [1]:
!nvidia-smi

Thu Jun 11 19:28:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip --quiet install transformers
!pip --quiet install tokenizers

     |████████████████████████████████| 675kB 8.2MB/s 
     |████████████████████████████████| 890kB 23.8MB/s 
     |████████████████████████████████| 3.8MB 43.6MB/s 
     |████████████████████████████████| 1.1MB 40.4MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r '/content/drive/My Drive/Colab Notebooks/Tweet Sentiment Extraction/Scripts/.' .

In [0]:
COLAB_BASE_PATH = '/content/drive/My Drive/Colab Notebooks/Tweet Sentiment Extraction/'
MODEL_BASE_PATH = COLAB_BASE_PATH + 'Models/Files/244-roBERTa_base/'

## Dependencies

In [6]:
import json, glob, warnings
from tweet_utility_scripts import *
from tweet_utility_preprocess_roberta_scripts_aux import *
from transformers import TFRobertaModel, RobertaConfig
from tokenizers import ByteLevelBPETokenizer
from tensorflow.keras import layers
from tensorflow.keras.models import Model

SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")
pd.set_option('max_colwidth', 120)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Load data

In [0]:
# Unzip files
!tar -xf '/content/drive/My Drive/Colab Notebooks/Tweet Sentiment Extraction/Data/complete_64_clean/fold_1.tar.gz'
!tar -xf '/content/drive/My Drive/Colab Notebooks/Tweet Sentiment Extraction/Data/complete_64_clean/fold_2.tar.gz'
!tar -xf '/content/drive/My Drive/Colab Notebooks/Tweet Sentiment Extraction/Data/complete_64_clean/fold_3.tar.gz'
!tar -xf '/content/drive/My Drive/Colab Notebooks/Tweet Sentiment Extraction/Data/complete_64_clean/fold_4.tar.gz'
!tar -xf '/content/drive/My Drive/Colab Notebooks/Tweet Sentiment Extraction/Data/complete_64_clean/fold_5.tar.gz'

In [8]:
database_base_path = COLAB_BASE_PATH + 'Data/complete_64_clean/'
k_fold = pd.read_csv(database_base_path + '5-fold.csv')
display(k_fold.head())

,textID,text,selected_text,sentiment,jaccard,text_len,text_wordCnt,text_tokenCnt,selected_text_len,selected_text_wordCnt,selected_text_tokenCnt,fold_1,fold_2,fold_3,fold_4,fold_5,imputed
0,cb774db0d1,"i`d have responded, if i were going","i`d have responded, if i were going",neutral,1.000000,35,7,10,35,7,10,train,validation,train,train,train,False
1,549e992a42,sooo sad i will miss you here in san diego!!!,sooo sad,negative,0.200000,45,10,13,8,2,3,validation,train,train,train,train,False
2,088c60f138,my boss is bullying me...,bullying me,negative,0.166667,25,5,6,11,2,2,train,train,train,validation,train,False
3,9642c003ef,what interview! leave me alone,leave me alone,negative,0.600000,30,5,6,14,3,3,train,train,train,validation,train,False
4,358bd9e861,"sons of ****, why couldn`t they put them on the releases we already bought","sons of ****,",negative,0.214286,74,14,17,13,3,4,train,train,train,validation,train,False


# Model parameters

In [9]:
vocab_path = COLAB_BASE_PATH + 'qa-transformers/roberta/roberta-base-vocab.json'
merges_path = COLAB_BASE_PATH + 'qa-transformers/roberta/roberta-base-merges.txt'
base_path = COLAB_BASE_PATH + 'qa-transformers/roberta/'

with open(MODEL_BASE_PATH + 'config.json') as json_file:
     config = json.load(json_file)

config

{'BATCH_SIZE': 32,
 'EPOCHS': 2,
 'ES_PATIENCE': 2,
 'LEARNING_RATE': 0.0001,
 'MAX_LEN': 64,
 'N_FOLDS': 5,
 'base_model_path': '/content/drive/My Drive/Colab Notebooks/Tweet Sentiment Extraction/qa-transformers/roberta/roberta-base-tf_model.h5',
 'config_path': '/content/drive/My Drive/Colab Notebooks/Tweet Sentiment Extraction/qa-transformers/roberta/roberta-base-config.json',
 'question_size': 4}

# Tokenizer

In [0]:
tokenizer = ByteLevelBPETokenizer(vocab_file=vocab_path, merges_file=merges_path, 
                                  lowercase=True, add_prefix_space=True)

# Model

In [0]:
module_config = RobertaConfig.from_pretrained(config['config_path'], output_hidden_states=False)

def model_fn(MAX_LEN):
    input_ids = layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_ids')
    attention_mask = layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name='attention_mask')
    
    base_model = TFRobertaModel.from_pretrained(config['base_model_path'], config=module_config, name="base_model")
    last_hidden_state, _  = base_model({'input_ids': input_ids, 'attention_mask': attention_mask})
    
    logits = layers.Dense(2, name="qa_outputs", use_bias=False)(last_hidden_state)
    
    start_logits, end_logits = tf.split(logits, 2, axis=-1)
    start_logits = tf.squeeze(start_logits, axis=-1)
    end_logits = tf.squeeze(end_logits, axis=-1)
    
    model = Model(inputs=[input_ids, attention_mask], outputs=[start_logits, end_logits])
    
    return model

# Make predictions

In [0]:
def predict_eval_df_clean(df, model, x_train, x_valid, dataset_fn, decode_fn, n_fold, tokenizer, config, question_size=None):
    fold_col = f'fold_{n_fold}'
    end_col = f'end_fold_{n_fold}'
    start_col = f'start_fold_{n_fold}'
    pred_col = f'prediction_fold_{n_fold}'
    jaccard_col = f'jaccard_fold_{n_fold}'
    
    train_preds = model.predict(dataset_fn(x_train, config['BATCH_SIZE']))
    valid_preds = model.predict(dataset_fn(x_valid, config['BATCH_SIZE']))
    
    df.loc[df[fold_col] == 'train', start_col] = train_preds[0].argmax(axis=-1)
    df.loc[df[fold_col] == 'train', end_col] = train_preds[1].argmax(axis=-1)
    df.loc[df[fold_col] == 'validation', start_col] = valid_preds[0].argmax(axis=-1)
    df.loc[df[fold_col] == 'validation', end_col] = valid_preds[1].argmax(axis=-1)
    
    df[end_col] = df[end_col].astype(int)
    df[start_col] = df[start_col].astype(int)
    if question_size is None:
        df[pred_col] = df.apply(lambda x: decode_fn(x[start_col], x[end_col], x['text'], tokenizer), axis=1)
    else:
        df[pred_col] = df.apply(lambda x: decode_fn(x[start_col], x[end_col], x['text'], config['question_size'], tokenizer), axis=1)

In [0]:
for n_fold in range(config['N_FOLDS']):
    n_fold +=1

    # Load data                               
    base_data_path = 'fold_%d/' % (n_fold)
    x_train = np.load(base_data_path + 'x_train.npy')
    y_train = np.load(base_data_path + 'y_train.npy')
    x_valid = np.load(base_data_path + 'x_valid.npy')
    y_valid = np.load(base_data_path + 'y_valid.npy')

    # Load model
    model_path = 'model_fold_%d.h5' % (n_fold)
    model = model_fn(config['MAX_LEN'])
    
    # Make predictions
    model.load_weights(MODEL_BASE_PATH + model_path)
    predict_eval_df_clean(k_fold, model, x_train, x_valid, get_test_dataset, decode, n_fold, tokenizer, config, config['question_size'])

# Model evaluation

In [0]:
#@title
def post_process(text, selected_text):
    text_words = text.lower().split()
    label_words = selected_text.lower().split()
    label = []

    for label_word in label_words:
        if label_word in text_words:
            label.append(label_word)
        else:
            best_match, best_ratio = '', 0
            for text_word in text_words:
                start_idx = text_word.find(label_word)
                if start_idx != -1:
                    end_idx = start_idx + len(label_word)
                    ratio = len(text_word[start_idx:end_idx]) / len(text_word)
                    if ratio > best_ratio:
                        best_ratio = ratio
                        best_match = text_word
            label.append(best_match)
            
    return ' '.join(label)


def post_process2(text, selected_text):
    text = text.lower().strip()
    selected_text = selected_text.lower().strip()
    
    idx_start, idx_end = None, None
    for index in (i for i, c in enumerate(text) if c == selected_text[0]):
        if text[index:index + len(selected_text)] == selected_text:
            idx_start = index
            idx_end = index + len(selected_text)
            break
    
    text_words = text.split()
    label_words = selected_text.split()
    start_label = label_words[0]
    end_label = label_words[-1]
    
    # clean start
    if start_label not in text_words:
        if len(label_words) > 1:
            search_start_idx = len(text_words)
            for idx, text_word in enumerate(text_words):
                if text_word == label_words[1]:
                    search_start_idx = idx
                    break
                    
            search_words = text_words[:search_start_idx]
        else:
            search_words = text_words
            
        best_match, best_ratio, best_index = '', 0, 0
        for text_word in search_words:
            start_idx = text_word.find(start_label)
            if start_idx != -1:
                end_idx = start_idx + len(start_label)
                ratio = len(text_word[start_idx:end_idx]) / len(text_word)
                if ratio > best_ratio:
                    best_ratio = ratio
                    best_match = text_word
                    best_index = start_idx
        idx_start = max(0, idx_start - best_index)
        
    # clean end
    if end_label not in text_words:
        if len(label_words) > 1:
            for idx, text_word in enumerate(text_words):
                if text_word == label_words[-2]:
                    break
            search_words = text_words[idx:]
        else:
            search_words = text_words
            
        best_match, best_ratio, best_index = '', 0, 0
        for text_word in search_words:
            start_idx = text_word.find(end_label)
            if start_idx != -1:
                end_idx = start_idx + len(end_label)
                ratio = len(text_word[start_idx:end_idx]) / len(text_word)
                if ratio > best_ratio:
                    best_ratio = ratio
                    best_match = text_word
                    best_index = start_idx
        idx_end = min(len(text), idx_end + (len(best_match) - len(end_label)))
        
    return text[idx_start:idx_end]


def post_process3(text, selected_text):
    text = text.lower().strip()
    selected_text = selected_text.lower().strip()
    
    text_words = text.split()
    label_words = selected_text.split()
    start_label = label_words[0]
    end_label = label_words[-1]
    
    if not start_label in text_words:
        if len(label_words) > 1:
            search_start_idx = len(text_words)
            for idx, text_word in enumerate(text_words):
                if text_word == label_words[1]:
                    search_start_idx = idx
                    break
            search_words = text_words[:search_start_idx]
        else:
            search_words = text_words
            
        best_match, best_ratio = '', 0
        for text_word in search_words:
            start_idx = text_word.find(start_label)
            if start_idx != -1:
                end_idx = start_idx + len(start_label)
                ratio = len(text_word[start_idx:end_idx]) / len(text_word)
                if ratio > best_ratio:
                    best_ratio = ratio
                    best_match = text_word
        label_words[0] = best_match
        
    
    if (len(label_words) > 1) & (not end_label in text_words):
        if len(label_words) > 1:
            for idx, text_word in enumerate(text_words):
                if text_word == label_words[-2]:
                    break
            search_words = text_words[idx:]
        else:
            search_words = text_words
            
        best_match, best_ratio = '', 0
        for text_word in search_words:
            start_idx = text_word.find(end_label)
            if start_idx != -1:
                end_idx = start_idx + len(end_label)
                ratio = len(text_word[start_idx:end_idx]) / len(text_word)
                if ratio > best_ratio:
                    best_ratio = ratio
                    best_match = text_word
        label_words[-1] = best_match
        
    label = ' '.join(label_words)
    
    return label

import re
from string import punctuation

punctuations = punctuation.replace('`', '')

def clean_punctuation(text):
    for punctuation in punctuations:
        text = text.replace(f'{punctuation}', ' ')

    return text.strip()


def post_process4(text, selected_text):
    text = text.lower().strip()
    selected_text = selected_text.lower().strip()
    
    text_words = text.split()
    label_words = selected_text.split()
    start_label = label_words[0]
    end_label = label_words[-1]
    
    if not start_label in text_words:
        if len(label_words) > 1:
            search_start_idx = len(text_words)
            for idx, text_word in enumerate(text_words):
                if text_word == label_words[1]:
                    search_start_idx = idx
                    break
                    
            search_words = text_words[:search_start_idx]
        else:
            search_words = text_words
        search_words = clean_punctuation(' '.join(search_words)).split()
        start_label = clean_punctuation(start_label)
            
        best_match, best_ratio = '', 0
        for text_word in search_words:
            start_idx = text_word.find(start_label)
            if start_idx != -1:
                end_idx = start_idx + len(start_label)
                ratio = len(text_word[start_idx:end_idx]) / len(text_word)
                if ratio > best_ratio:
                    best_ratio = ratio
                    best_match = text_word
        label_words[0] = best_match
        
    
    if (len(label_words) > 1) & (not end_label in text_words):
        if len(label_words) > 1:
            for idx, text_word in enumerate(text_words):
                if text_word == label_words[-2]:
                    break
            search_words = text_words[idx:]
        else:
            searche_words = text_words
        search_words = clean_punctuation(' '.join(search_words)).split()
        end_label = clean_punctuation(end_label)
            
        best_match, best_ratio = '', 0
        for text_word in search_words:
            start_idx = text_word.find(end_label)
            if start_idx != -1:
                end_idx = start_idx + len(end_label)
                ratio = len(text_word[start_idx:end_idx]) / len(text_word)
                if ratio > best_ratio:
                    best_ratio = ratio
                    best_match = text_word
        label_words[-1] = best_match
        
    label = ' '.join(label_words)
    
    return label

In [52]:
#@title
display(evaluate_model_kfold(k_fold, config['N_FOLDS']).head(4).style.applymap(color_map))

print(f'\nPost-process standard')
k_fold_post = k_fold.copy()
for n_fold in range(config['N_FOLDS']):
    n_fold +=1
    pred_col = f'prediction_fold_{n_fold}'

    k_fold_post[pred_col] = k_fold_post.apply(lambda x: ' '.join([word for word in x[pred_col].split() if word in x['text'].split()]), axis=1)
    k_fold_post[pred_col] = k_fold_post.apply(lambda x: x['text'] if (x[pred_col] == '') else x[pred_col], axis=1)
    k_fold_post[pred_col].fillna(k_fold_post['text'], inplace=True)

display(evaluate_model_kfold(k_fold_post, config['N_FOLDS']).head(4).style.applymap(color_map))

print(f'\nPost-process 1')
k_fold_post = k_fold.copy()
for n_fold in range(config['N_FOLDS']):
    n_fold +=1
    pred_col = f'prediction_fold_{n_fold}'

    k_fold_post[pred_col] = k_fold_post.apply(lambda x: post_process(x['text'], x[pred_col]), axis=1)
    k_fold_post[pred_col] = k_fold_post.apply(lambda x: x['text'] if (x[pred_col] == '') else x[pred_col], axis=1)
    k_fold_post[pred_col].fillna(k_fold_post['text'], inplace=True)

display(evaluate_model_kfold(k_fold_post, config['N_FOLDS']).head(4).style.applymap(color_map))

print(f'\nPost-process 2')
k_fold_post = k_fold.copy()
for n_fold in range(config['N_FOLDS']):
    n_fold +=1
    pred_col = f'prediction_fold_{n_fold}'

    k_fold_post[pred_col] = k_fold_post.apply(lambda x: x['text'] if (x[pred_col] == '') else x[pred_col], axis=1)
    k_fold_post[pred_col].fillna(k_fold_post['text'], inplace=True)
    k_fold_post[pred_col] = k_fold_post.apply(lambda x: post_process2(x['text'], x[pred_col]), axis=1)
    k_fold_post[pred_col] = k_fold_post.apply(lambda x: x['text'] if (x[pred_col] == '') else x[pred_col], axis=1)
    k_fold_post[pred_col].fillna(k_fold_post['text'], inplace=True)

display(evaluate_model_kfold(k_fold_post, config['N_FOLDS']).head(4).style.applymap(color_map))

print(f'\nPost-process 3')
k_fold_post = k_fold.copy()
for n_fold in range(config['N_FOLDS']):
    n_fold +=1
    pred_col = f'prediction_fold_{n_fold}'

    k_fold_post[pred_col] = k_fold_post.apply(lambda x: x['text'] if (x[pred_col] == '') else x[pred_col], axis=1)
    k_fold_post[pred_col].fillna(k_fold_post['text'], inplace=True)
    k_fold_post[pred_col] = k_fold_post.apply(lambda x: post_process3(x['text'], x[pred_col]), axis=1)
    k_fold_post[pred_col] = k_fold_post.apply(lambda x: x['text'] if (x[pred_col] == '') else x[pred_col], axis=1)
    k_fold_post[pred_col].fillna(k_fold_post['text'], inplace=True)

display(evaluate_model_kfold(k_fold_post, config['N_FOLDS']).head(4).style.applymap(color_map))

print(f'\nPost-process 4')
k_fold_post = k_fold.copy()
for n_fold in range(config['N_FOLDS']):
    n_fold +=1
    pred_col = f'prediction_fold_{n_fold}'

    k_fold_post[pred_col] = k_fold_post.apply(lambda x: x['text'] if (x[pred_col] == '') else x[pred_col], axis=1)
    k_fold_post[pred_col].fillna(k_fold_post['text'], inplace=True)
    k_fold_post[pred_col] = k_fold_post.apply(lambda x: post_process4(x['text'], x[pred_col]), axis=1)
    k_fold_post[pred_col] = k_fold_post.apply(lambda x: x['text'] if (x[pred_col] == '') else x[pred_col], axis=1)
    k_fold_post[pred_col].fillna(k_fold_post['text'], inplace=True)

display(evaluate_model_kfold(k_fold_post, config['N_FOLDS']).head(4).style.applymap(color_map))

,Train,Valid,Var,Train_fold_1,Valid_fold_1,Train_fold_2,Valid_fold_2,Train_fold_3,Valid_fold_3,Train_fold_4,Valid_fold_4,Train_fold_5,Valid_fold_5
Metric/Jaccard,,,,,,,,,,,,,
Overall,0.763745,0.724635,0.039110,0.768143,0.726145,0.766966,0.722700,0.754963,0.717429,0.764101,0.730707,0.764553,0.726194
Sentiment negative,0.621581,0.546681,0.074900,0.631965,0.545604,0.628694,0.543692,0.601063,0.546100,0.621861,0.549266,0.624321,0.548743
Sentiment neutral,0.977574,0.976570,0.001003,0.977815,0.975470,0.977186,0.976501,0.977526,0.976888,0.977893,0.976896,0.977449,0.977096
Sentiment positive,0.605819,0.548022,0.057797,0.610390,0.555284,0.610374,0.544451,0.595937,0.524793,0.606279,0.565009,0.606114,0.550573



Post-process standard


,Train,Valid,Var,Train_fold_1,Valid_fold_1,Train_fold_2,Valid_fold_2,Train_fold_3,Valid_fold_3,Train_fold_4,Valid_fold_4,Train_fold_5,Valid_fold_5
Metric/Jaccard,,,,,,,,,,,,,
Overall,0.765828,0.726872,0.038956,0.770495,0.727576,0.769097,0.724737,0.756732,0.719919,0.766087,0.732851,0.766727,0.729277
Sentiment negative,0.625010,0.550889,0.074121,0.635645,0.548708,0.632060,0.547415,0.604129,0.550706,0.625041,0.553621,0.628174,0.553993
Sentiment neutral,0.978185,0.977246,0.000938,0.978402,0.976348,0.977993,0.977153,0.978108,0.977133,0.978511,0.977439,0.977908,0.978159
Sentiment positive,0.608654,0.550567,0.058087,0.613904,0.555941,0.613158,0.546819,0.598126,0.528375,0.609019,0.567298,0.609066,0.554404



Post-process 1


,Train,Valid,Var,Train_fold_1,Valid_fold_1,Train_fold_2,Valid_fold_2,Train_fold_3,Valid_fold_3,Train_fold_4,Valid_fold_4,Train_fold_5,Valid_fold_5
Metric/Jaccard,,,,,,,,,,,,,
Overall,0.764923,0.726156,0.038768,0.769304,0.727264,0.768293,0.724135,0.756095,0.718643,0.765272,0.732221,0.765651,0.728515
Sentiment negative,0.622937,0.548830,0.074107,0.633016,0.548278,0.630021,0.544481,0.602712,0.547818,0.623369,0.551721,0.625566,0.551852
Sentiment neutral,0.978338,0.977465,0.000873,0.978562,0.976723,0.978265,0.977292,0.978185,0.977201,0.978711,0.977473,0.977969,0.978637
Sentiment positive,0.607389,0.549812,0.057577,0.612205,0.554813,0.612033,0.547332,0.597235,0.526760,0.607619,0.566930,0.607856,0.553226


...

,Train,Valid,Var,Train_fold_1,Valid_fold_1,Train_fold_2,Valid_fold_2,Train_fold_3,Valid_fold_3,Train_fold_4,Valid_fold_4,Train_fold_5,Valid_fold_5
Metric/Jaccard,,,,,,,,,,,,,
Overall,0.764911,0.726162,0.038749,0.769390,0.727060,0.768233,0.724300,0.756026,0.718755,0.765232,0.732221,0.765674,0.728475
Sentiment negative,0.622718,0.548708,0.074010,0.632888,0.547166,0.629733,0.544991,0.602413,0.548036,0.623022,0.551795,0.625534,0.551553
Sentiment neutral,0.978308,0.977439,0.000869,0.978530,0.976723,0.978224,0.977292,0.978153,0.977201,0.978697,0.977344,0.977936,0.978637
Sentiment positive,0.607589,0.549979,0.057610,0.612643,0.555158,0.612153,0.547408,0.597326,0.526926,0.607822,0.567034,0.608002,0.553368



Post-process 3


,Train,Valid,Var,Train_fold_1,Valid_fold_1,Train_fold_2,Valid_fold_2,Train_fold_3,Valid_fold_3,Train_fold_4,Valid_fold_4,Train_fold_5,Valid_fold_5
Metric/Jaccard,,,,,,,,,,,,,
Overall,0.764910,0.726150,0.038760,0.769285,0.727282,0.768272,0.724158,0.756074,0.718710,0.765246,0.732221,0.765671,0.728378
Sentiment negative,0.622923,0.548807,0.074115,0.632973,0.548411,0.629986,0.544481,0.602682,0.547912,0.623297,0.551721,0.625677,0.551512
Sentiment neutral,0.978339,0.977465,0.000874,0.978562,0.976723,0.978265,0.977292,0.978185,0.977201,0.978715,0.977473,0.977969,0.978637
Sentiment positive,0.607358,0.549814,0.057544,0.612183,0.554753,0.611996,0.547408,0.597195,0.526891,0.607594,0.566930,0.607821,0.553090



Post-process 4


,Train,Valid,Var,Train_fold_1,Valid_fold_1,Train_fold_2,Valid_fold_2,Train_fold_3,Valid_fold_3,Train_fold_4,Valid_fold_4,Train_fold_5,Valid_fold_5
Metric/Jaccard,,,,,,,,,,,,,
Overall,0.765244,0.725998,0.039247,0.769939,0.727244,0.768536,0.723179,0.756341,0.719082,0.765290,0.731836,0.766116,0.728647
Sentiment negative,0.624309,0.549442,0.074867,0.634772,0.548894,0.631802,0.544055,0.603712,0.549824,0.624328,0.551603,0.626930,0.552831
Sentiment neutral,0.977874,0.976909,0.000965,0.978157,0.975792,0.977595,0.976541,0.977774,0.977059,0.978177,0.977298,0.977665,0.977854
Sentiment positive,0.607813,0.549492,0.058322,0.613217,0.555440,0.612106,0.545623,0.597680,0.526555,0.607526,0.566022,0.608537,0.553818


## Post-processing evaluation

In [53]:
#@title
k_fold_post = k_fold.copy()
k_fold_post.loc[k_fold_post['sentiment'] == 'neutral', 'selected_text'] = k_fold_post["text"]
print('\nImpute neutral')
display(evaluate_model_kfold(k_fold_post, config['N_FOLDS']).head(1).style.applymap(color_map))

k_fold_post = k_fold.copy()
k_fold_post.loc[k_fold_post['text_wordCnt'] <= 3, 'selected_text'] = k_fold_post["text"]
print('\nImpute <= 3')
display(evaluate_model_kfold(k_fold_post, config['N_FOLDS']).head(1).style.applymap(color_map))

k_fold_post = k_fold.copy()
k_fold_post['selected_text'] = k_fold_post['selected_text'].apply(lambda x: x.replace('!!!!', '!') if len(x.split())==1 else x)
k_fold_post['selected_text'] = k_fold_post['selected_text'].apply(lambda x: x.replace('..', '.') if len(x.split())==1 else x)
k_fold_post['selected_text'] = k_fold_post['selected_text'].apply(lambda x: x.replace('...', '.') if len(x.split())==1 else x)
print('\nImpute noise')
display(evaluate_model_kfold(k_fold_post, config['N_FOLDS']).head(1).style.applymap(color_map))

k_fold_post = k_fold.copy()
k_fold_post.loc[k_fold_post['sentiment'] == 'neutral', 'selected_text'] = k_fold_post["text"]
k_fold_post.loc[k_fold_post['text_wordCnt'] <= 3, 'selected_text'] = k_fold_post["text"]
print('\nImpute neutral and <= 3')
display(evaluate_model_kfold(k_fold_post, config['N_FOLDS']).head(1).style.applymap(color_map))

k_fold_post = k_fold.copy()
k_fold_post.loc[k_fold_post['sentiment'] == 'neutral', 'selected_text'] = k_fold_post["text"]
k_fold_post.loc[k_fold_post['text_wordCnt'] <= 3, 'selected_text'] = k_fold_post["text"]
k_fold_post['selected_text'] = k_fold_post['selected_text'].apply(lambda x: x.replace('!!!!', '!') if len(x.split())==1 else x)
k_fold_post['selected_text'] = k_fold_post['selected_text'].apply(lambda x: x.replace('..', '.') if len(x.split())==1 else x)
k_fold_post['selected_text'] = k_fold_post['selected_text'].apply(lambda x: x.replace('...', '.') if len(x.split())==1 else x)
print('\nImpute neutral and <= 3 and mpute noise')
display(evaluate_model_kfold(k_fold_post, config['N_FOLDS']).head(1).style.applymap(color_map))


Impute neutral


,Train,Valid,Var,Train_fold_1,Valid_fold_1,Train_fold_2,Valid_fold_2,Train_fold_3,Valid_fold_3,Train_fold_4,Valid_fold_4,Train_fold_5,Valid_fold_5
Metric/Jaccard,,,,,,,,,,,,,
Overall,0.770644,0.732168,0.038476,0.774838,0.733522,0.773799,0.730291,0.762175,0.725197,0.770991,0.738243,0.771417,0.733588



Impute <= 3


,Train,Valid,Var,Train_fold_1,Valid_fold_1,Train_fold_2,Valid_fold_2,Train_fold_3,Valid_fold_3,Train_fold_4,Valid_fold_4,Train_fold_5,Valid_fold_5
Metric/Jaccard,,,,,,,,,,,,,
Overall,0.762805,0.724628,0.038178,0.765693,0.724518,0.767031,0.725257,0.756310,0.717088,0.761670,0.730087,0.763324,0.726188



Impute noise


,Train,Valid,Var,Train_fold_1,Valid_fold_1,Train_fold_2,Valid_fold_2,Train_fold_3,Valid_fold_3,Train_fold_4,Valid_fold_4,Train_fold_5,Valid_fold_5
Metric/Jaccard,,,,,,,,,,,,,
Overall,0.762431,0.723521,0.038910,0.766781,0.725607,0.765678,0.721150,0.753806,0.715981,0.762639,0.729498,0.763251,0.725369



Impute neutral and <= 3


,Train,Valid,Var,Train_fold_1,Valid_fold_1,Train_fold_2,Valid_fold_2,Train_fold_3,Valid_fold_3,Train_fold_4,Valid_fold_4,Train_fold_5,Valid_fold_5
Metric/Jaccard,,,,,,,,,,,,,
Overall,0.769465,0.731916,0.037550,0.772209,0.731508,0.773673,0.732383,0.763262,0.724856,0.768269,0.737437,0.769913,0.733396



Impute neutral and <= 3 and mpute noise


,Train,Valid,Var,Train_fold_1,Valid_fold_1,Train_fold_2,Valid_fold_2,Train_fold_3,Valid_fold_3,Train_fold_4,Valid_fold_4,Train_fold_5,Valid_fold_5
Metric/Jaccard,,,,,,,,,,,,,
Overall,0.768182,0.730814,0.037368,0.770893,0.730970,0.772385,0.730895,0.762121,0.723408,0.766854,0.736228,0.768658,0.732571


# Error analysis

## 10 worst predictions

In [0]:
#@title
k_fold['jaccard_mean'] = (k_fold['jaccard_fold_1'] + k_fold['jaccard_fold_2'] + 
                          k_fold['jaccard_fold_3'] + k_fold['jaccard_fold_4'] + 
                          k_fold['jaccard_fold_4']) / 5

display(k_fold[['text', 'selected_text', 'sentiment', 'jaccard', 'jaccard_mean', 
                'prediction_fold_1', 'prediction_fold_2', 'prediction_fold_3', 
                'prediction_fold_4', 'prediction_fold_5']].sort_values(by=['jaccard_mean']).head(10))

# Sentiment

In [0]:
#@title
print('\n sentiment == neutral')
display(k_fold[k_fold['sentiment'] == 'neutral'][['text', 'selected_text', 
                                                  'jaccard_mean', 'prediction_fold_1', 
                                                  'prediction_fold_2', 'prediction_fold_3', 
                                                  'prediction_fold_4', 'prediction_fold_5']].sort_values(by=['jaccard_mean']).head(10))

print('\n sentiment == positive')
display(k_fold[k_fold['sentiment'] == 'positive'][['text', 'selected_text', 
                                                   'jaccard_mean', 'prediction_fold_1', 
                                                   'prediction_fold_2', 'prediction_fold_3', 
                                                   'prediction_fold_4', 'prediction_fold_5']].sort_values(by=['jaccard_mean']).head(10))

print('\n sentiment == negative')
display(k_fold[k_fold['sentiment'] == 'negative'][['text', 'selected_text', 
                                                   'jaccard_mean', 'prediction_fold_1', 
                                                   'prediction_fold_2', 'prediction_fold_3', 
                                                   'prediction_fold_4', 'prediction_fold_5']].sort_values(by=['jaccard_mean']).head(10))

# text_tokenCnt

In [0]:
#@title
print('\n text_tokenCnt <= 3')
display(k_fold[k_fold['text_tokenCnt'] <= 3][['text', 'selected_text', 'sentiment', 
                                              'jaccard_mean', 'prediction_fold_1', 
                                              'prediction_fold_2', 'prediction_fold_3', 
                                              'prediction_fold_4', 'prediction_fold_5']].sort_values(by=['jaccard_mean']).head(5))

print('\n text_tokenCnt >= 45')
display(k_fold[k_fold['text_tokenCnt'] >= 45][['text', 'selected_text',  'sentiment', 
                                               'jaccard_mean', 'prediction_fold_1', 
                                               'prediction_fold_2', 'prediction_fold_3', 
                                               'prediction_fold_4', 'prediction_fold_5']].sort_values(by=['jaccard_mean']).head(5))

# selected_text_tokenCnt

In [0]:
#@title
print('\n selected_text_tokenCnt <= 3')
display(k_fold[k_fold['selected_text_tokenCnt'] <= 3][['text', 'selected_text',  'sentiment', 
                                                       'jaccard_mean', 'prediction_fold_1', 
                                                       'prediction_fold_2', 'prediction_fold_3', 
                                                       'prediction_fold_4', 'prediction_fold_5']].sort_values(by=['jaccard_mean']).head(5))
print('\n selected_text_tokenCnt >= 45')
display(k_fold[k_fold['selected_text_tokenCnt'] >= 45][['text', 'selected_text',  'sentiment', 
                                                        'jaccard_mean', 'prediction_fold_1', 
                                                        'prediction_fold_2', 'prediction_fold_3', 
                                                        'prediction_fold_4', 'prediction_fold_5']].sort_values(by=['jaccard_mean']).head(5))

# Jaccard histogram

In [0]:
#@title
fig, ax = plt.subplots(1, 1, figsize=(20, 5))
sns.distplot(k_fold['jaccard_mean'], ax=ax).set_title(f"Overall [{len(k_fold)}]")
sns.despine()
plt.show()

## By sentiment

In [0]:
#@title
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(20, 15), sharex=False)
sns.distplot(k_fold[k_fold['sentiment'] == 'neutral']['jaccard_mean'], ax=ax1).set_title(f"Neutral [{len(k_fold[k_fold['sentiment'] == 'neutral'])}]")
sns.distplot(k_fold[k_fold['sentiment'] == 'positive']['jaccard_mean'], ax=ax2).set_title(f"Positive [{len(k_fold[k_fold['sentiment'] == 'positive'])}]")
sns.distplot(k_fold[k_fold['sentiment'] == 'negative']['jaccard_mean'], ax=ax3).set_title(f"Negative [{len(k_fold[k_fold['sentiment'] == 'negative'])}]")
sns.despine()
plt.show()

## By text token count

In [0]:
#@title
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 10), sharex=False)
sns.distplot(k_fold[k_fold['text_tokenCnt'] <= 3]['jaccard_mean'], ax=ax1).set_title(f"text_tokenCnt <= 3 [{len(k_fold[k_fold['text_tokenCnt'] <= 3])}]")
sns.distplot(k_fold[k_fold['text_tokenCnt'] >= 45]['jaccard_mean'], ax=ax2).set_title(f"text_tokenCnt >= 45 [{len(k_fold[k_fold['text_tokenCnt'] >= 45])}]")
sns.despine()
plt.show()

## By selected_text token count

In [0]:
#@title
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 10), sharex=False)
sns.distplot(k_fold[k_fold['selected_text_tokenCnt'] <= 3]['jaccard_mean'], ax=ax1).set_title(f"selected_text_tokenCnt <= 3 [{len(k_fold[k_fold['selected_text_tokenCnt'] <= 3])}]")
sns.distplot(k_fold[k_fold['selected_text_tokenCnt'] >= 45]['jaccard_mean'], ax=ax2).set_title(f"selected_text_tokenCnt >= 45 [{len(k_fold[k_fold['selected_text_tokenCnt'] >= 45])}]")
sns.despine()
plt.show()